# All imports and inits

In [39]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from pinecone import Pinecone, ServerlessSpec
import os
import requests
import PyPDF2
import textwrap
import numpy as np
from typing import List, Dict, Tuple, Optional

# Important: Import pinecone-client properly
# Load environment variables from .env file
load_dotenv()

DATA_PATH = os.getenv("DATA_PATH")
PINECONE_API = os.getenv("PINECONE_API")
PINECONE_ENV = os.getenv("PINECONE_ENV")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print("PINECONE_API", PINECONE_API)



# Groq API settings
GROQ_EMBED_URL = "https://api.groq.com/openai/v1/embeddings"
GROQ_CHAT_URL = "https://api.groq.com/openai/v1/chat/completions"
EMBEDDING_MODEL = "llama3-405b-8192-embed"
LLM_MODEL = "llama3-70b-8192"


# Configure headers for Groq API requests
GROQ_HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}


ModuleNotFoundError: No module named 'PyPDF2'

# PDF loader

In [31]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()


documents = load_documents()
documents[0]


# def extract_text_from_pdf(pdf_path: str) -> str:
#     """Extract text from a PDF file."""
#     with open(pdf_path, 'r') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         text = ""
#         for page_num in range(len(pdf_reader.pages)):
#             page = pdf_reader.pages[page_num]
#             text += page.extract_text() + "\n"
#     return text
# extract_text_from_pdf(DATA_PATH)


Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEnc

Document(metadata={'producer': 'Nitro Pro  (11. 0. 1. 10)', 'creator': 'Nitro Pro  (11. 0. 1. 10)', 'creationdate': '2025-03-09T10:44:04+00:00', 'moddate': '2025-03-09T15:44:28+05:00', 'title': 'PowerPoint Presentation', 'author': 'James Kurose', 'source': 'D:\\Disrupt Labs\\Rag Experiments\\env\\Rag-pipelines-experiments\\data\\4.1_video_slides.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Network Layer:\nData Plane\n\uf0a7 Overview of Network Layer\n\uf0a7 What’s Inside a Router?\n\uf0a7 The Internet Protocol: IPv4, Addressing, NAT\nIPv6\n\uf0a7 Generalized Forwarding and SDN\n\uf0a7 Middleboxes\n\uf0a7 Summary\nCOMPSCI 453 Computer Networks\nProfessor Jim Kurose\nCollege of Information and Computer Sciences\nUniversity of Massachusetts\nClass textbook:\nComputer Networking: A Top-\nDown Approach (8th ed.)\nJ.F. Kurose, K.W . Ross\nPearson, 2020\nhttp://gaia.cs.umass.edu/kurose_ross')

# Text Splitting \ Chunking using Langchain

In [32]:


def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size = 800,
        chunk_overlap  = 80,
        length_function = len,
        is_separator_regex = False # considers separators like '\n\n'if true
    )
    docs = text_splitter.split_documents(documents)
    return docs


chunks = split_documents(documents)
chunks[0]


Document(metadata={'producer': 'Nitro Pro  (11. 0. 1. 10)', 'creator': 'Nitro Pro  (11. 0. 1. 10)', 'creationdate': '2025-03-09T10:44:04+00:00', 'moddate': '2025-03-09T15:44:28+05:00', 'title': 'PowerPoint Presentation', 'author': 'James Kurose', 'source': 'D:\\Disrupt Labs\\Rag Experiments\\env\\Rag-pipelines-experiments\\data\\4.1_video_slides.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Network Layer:\nData Plane\n\uf0a7 Overview of Network Layer\n\uf0a7 What’s Inside a Router?\n\uf0a7 The Internet Protocol: IPv4, Addressing, NAT\nIPv6\n\uf0a7 Generalized Forwarding and SDN\n\uf0a7 Middleboxes\n\uf0a7 Summary\nCOMPSCI 453 Computer Networks\nProfessor Jim Kurose\nCollege of Information and Computer Sciences\nUniversity of Massachusetts\nClass textbook:\nComputer Networking: A Top-\nDown Approach (8th ed.)\nJ.F. Kurose, K.W . Ross\nPearson, 2020\nhttp://gaia.cs.umass.edu/kurose_ross')

# Init Pinecone

In [40]:
# Configure Pinecone
def init_pinecone(index_name: str) -> None:
    """Initialize Pinecone and create index if it doesn't exist."""
    pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
    
    # Check if index exists, if not create it
    if index_name not in pinecone.list_indexes():
        pinecone.create_index(
            name=index_name,
            dimension=1024,  # Updated to match your index dimension
            metric="cosine",
            spec={"serverless": {"cloud": "aws", "region": "us-east-1"}}  # Updated to match your configuration
        )
    return pinecone.Index(index_name)

init_pinecone("test-index")


NameError: name 'pinecone' is not defined

# Creating Embeddings Via OpenAI API of groq inference


In [35]:
def get_embedding(text: str):
    """Get embedding for a text using Groq API."""
    response = requests.post(
        GROQ_EMBED_URL,
        headers=GROQ_HEADERS,
        json={
            "model": EMBEDDING_MODEL,
            "input": text
        }
    )
    
    if response.status_code != 200:
        raise Exception(f"Error getting embedding: {response.text}")
    
    # Get embedding and truncate to match Pinecone dimension if needed
    embedding = response.json()["data"][0]["embedding"]
    if len(embedding) > 1024:  # Ensure embedding matches your Pinecone dimension
        embedding = embedding[:1024]
        
    return embedding


# Upsert to Pinecone

In [ ]:
def upsert_chunks_to_pinecone(index, chunks: List[str], pdf_name: str) -> None:
    """Convert chunks to embeddings and upsert to Pinecone."""
    vectors = []
    
    for i, chunk in enumerate(chunks):
        # Progress indicator
        print(f"Processing chunk {i+1}/{len(chunks)}...")
        
        embedding = get_embedding(chunk)
        vectors.append({
            "id": f"{pdf_name}-chunk-{i}",
            "values": embedding,
            "metadata": {
                "text": chunk,
                "source": pdf_name,
                "chunk_id": i
            }
        })
        
        # Batch upsert every 100 vectors or at the end
        if len(vectors) >= 100 or i == len(chunks) - 1:
            index.upsert(vectors=vectors)
            vectors = []
    
    print(f"Successfully uploaded {len(chunks)} chunks to Pinecone.")


# Query Groq

In [ ]:
def query_groq(prompt: str) -> str:
    """Query Groq LLM with a prompt."""
    response = requests.post(
        GROQ_CHAT_URL,
        headers=GROQ_HEADERS,
        json={
            "model": LLM_MODEL,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1,
            "max_tokens": 1024
        }
    )
    
    if response.status_code != 200:
        raise Exception(f"Error querying Groq: {response.text}")
    
    return response.json()["choices"][0]["message"]["content"]


# Search Pinecone


In [ ]:
def search_pinecone(index, query: str, top_k: int = 5) -> List[Dict]:
    """Search Pinecone for similar chunks based on the query."""
    query_embedding = get_embedding(query)
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    return results.matches


# Generate Response

In [ ]:
def generate_response(query: str, context: List[Dict]) -> str:
    """Generate a response based on the query and retrieved context."""
    # Create a prompt with context
    context_text = "\n\n".join([match["metadata"]["text"] for match in context])
    
    prompt = f"""You are a helpful assistant that answers questions based on provided context. 
    
    CONTEXT:
    {context_text}
    
    QUESTION:
    {query}
    
    INSTRUCTIONS:
    - Answer the question based only on the provided context
    - If the answer is not in the context, say "I don't have enough information to answer that question."
    - Cite specific parts of the context that support your answer
    - Be concise and clear
    
    ANSWER:"""
    
    return query_groq(prompt)


# Process pdf document


In [ ]:
def process_pdf_document(pdf_path: str, index_name: str) -> None:
    """Process a PDF document and store in Pinecone."""
    # Initialize Pinecone
    index = init_pinecone(index_name)
    
    # Extract text from PDF
    print(f"Extracting text from {pdf_path}...")
    pdf_text = extract_text_from_pdf(pdf_path)
    
    # Split text into chunks
    print("Splitting text into chunks...")
    chunks = split_text(pdf_text)
    
    # Get PDF filename for identifying the source
    pdf_name = os.path.basename(pdf_path)
    
    # Upsert chunks to Pinecone
    print(f"Upserting {len(chunks)} chunks to Pinecone...")
    upsert_chunks_to_pinecone(index, chunks, pdf_name)
    
    print(f"Successfully processed {pdf_path} and stored in Pinecone index '{index_name}'.")
    

# Answer_query

In [ ]:
def answer_query(query: str, index_name: str) -> str:
    """Answer a query using the RAG pipeline."""
    # Initialize Pinecone
    index = init_pinecone(index_name)
    
    # Search for relevant chunks
    print(f"Searching for relevant context for: '{query}'")
    matches = search_pinecone(index, query)
    
    if not matches:
        return "No relevant information found in the documents."
    
    # Generate response using Groq LLM
    print("Generating response...")
    response = generate_response(query, matches)
    
    return response


# Main func

usage: ipykernel_launcher.py [-h] [--pdf PDF] [--query QUERY] [--index INDEX]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\Osama\AppData\Roaming\jupyter\runtime\kernel-v311f1b8fae0eaad2f829c880752cf0ae8a9213684.json


SystemExit: 2